## Constructing polynomial expansion

``chaospy`` relies on the package ``numpoly`` in the backend to represent all its polynomials.
It is used for the creation of polynomial expansion. like ``chaospy.orth_ttr``, and as the return object from the constructors like ``fit_regression`` and ``fit_quadrature``.

For a more extensive overview over the polynomial class, see the ``numpoly`` documentation:  https://numpoly.readthedocs.io

In [1]:
import chaospy
import numpy
numpy.seterr(all="ignore")

{'divide': 'warn', 'over': 'warn', 'under': 'ignore', 'invalid': 'warn'}

A simple polynomial can be created through ``variable`` constructor. For example to construct a simple bivariate polynomial:

In [2]:
q0, q1 = chaospy.variable(dims=2)
q0

polynomial(q0)

A collection of polynomial can be manipulated using basic arithmetic operators and joined together into polynomial expansions:

In [3]:
polynomials = chaospy.polynomial([1, q0, 1-q0*q1, q0**2*q1, q0-q1**2])
polynomials

polynomial([1, q0, 1-q0*q1, q0**2*q1, -q1**2+q0])

Note that constants and simple polynomials can be joined together into arrays without any problems.

In practice, having the ability to fine tune a polynomial exactly as one wants it can be useful, but it can also be cumbersome when dealing with larger arrays for application.
To automate the construction of simple polynomials, there is the ``basis`` constructor. In its simplest forms it creates an array of simple monomials:

In [4]:
chaospy.basis(4)

polynomial([1, q0, q0**2, q0**3, q0**4])

It can be expanded to include number of dimensions and a lower bound for the polynomial order:

In [5]:
chaospy.basis(start=1, stop=2, dim=2)

polynomial([q1, q0, q1**2, q0*q1, q0**2])

And as likely familiar elsewhere, there are constructors for the orthogonal polynomials as well:

In [6]:
distribution = chaospy.Normal(0, 1)
chaospy.orth_ttr(3, distribution)

polynomial([1.0, q0, -1.0+q0**2, -3.0*q0+q0**3])

### Join Univariate Expansions into One Multivariate Expansion

Often one have access to multiple univariate expansions and one wants to join them into a single one.
And preferably in a way where the user has a high level of control.

As an example, we start with two univeraite expansions that one wants to join:

In [7]:
expansions = [chaospy.basis(3), chaospy.orth_ttr(3, distribution)]
expansions

[polynomial([1, q0, q0**2, q0**3]),
 polynomial([1.0, q0, -1.0+q0**2, -3.0*q0+q0**3])]

First step is to ensure that each dimension have their own asigned dimension.
This can be done as follows:

In [8]:
variables = chaospy.variable(len(expansions))
expansions = [expansion(q0=variable) for expansion, variable in zip(expansions, variables)]
expansions

[polynomial([1, q0, q0**2, q0**3]),
 polynomial([1.0, q1, -1.0+q1**2, -3.0*q1+q1**3])]

To join the expansions, we use the function ``chaospy.bindex`` that creates indices following a polynomial truncation scheme.
The function allowes for the creation of indices that allows for the join of the various expansions:

See [Truncation of Polynomial Expansions](./truncation.ipynb) for more details on the form of the truncation rules.

In [9]:
indices = chaospy.bindex(start=0, stop=4, dimensions=2)
indices.T

array([[0, 0, 1, 0, 1, 2, 0, 1, 2, 3],
       [0, 1, 0, 2, 1, 0, 3, 2, 1, 0]])

Joining the expansions can then be created by extracting polynomials from the expansions and joining them by multiplying them together:

In [10]:
expansion = chaospy.prod([expansion[index] for expansion, index in zip(expansions, indices.T)], axis=0)
expansion

polynomial([1.0, q1, q0, -1.0+q1**2, q0*q1, q0**2, -3.0*q1+q1**3,
            -q0+q0*q1**2, q0**2*q1, q0**3])